In [29]:
# Model without CRF

from keras.utils.np_utils import to_categorical
#from tflearn.data_utils import to_categorical

In [30]:
from keras.models import load_model

In [31]:
import pandas as pd
import numpy as np
from keras.models import load_model


data = pd.read_csv('/home/sharath/postags_v1.csv', encoding='latin1', sep = ',')
data = data.fillna(method="ffill")
words = list(set(data["Word"].values))
words.append("ENDPAD")
n_words = len(words)
tags = list(set(data["Tag"].values))
n_tags = len(tags)

class SentenceGetter(object):
    
    def __init__(self, data):
        self.n_sent = 1
        self.data = data
        self.empty = False
        agg_func = lambda s: [(w, p, t) for w, p, t in zip(s["Word"].values.tolist(),
                                                           s["POS"].values.tolist(),
                                                           s["Tag"].values.tolist())]
        self.grouped = self.data.groupby("Sentence #").apply(agg_func)
        self.sentences = [s for s in self.grouped]
    
    def get_next(self):
        try:
            s = self.grouped["Sentence: {}".format(self.n_sent)]
            self.n_sent += 1
            return s
        except:
            return None

getter = SentenceGetter(data)
sent = getter.get_next()
sentences = getter.sentences
max_len = 50
word2idx = {w: i for i, w in enumerate(words)}
tag2idx = {t: i for i, t in enumerate(tags)}
from keras.preprocessing.sequence import pad_sequences
X = [[word2idx[w[0]] for w in s] for s in sentences]
X = pad_sequences(maxlen=max_len, sequences=X, padding="post", value=n_words - 1)
y = [[tag2idx[w[2]] for w in s] for s in sentences]
y = pad_sequences(maxlen=max_len, sequences=y, padding="post", value=tag2idx["O"])
#from keras.utils import to_categorical
y = [to_categorical(i, num_classes=n_tags) for i in y]
from sklearn.model_selection import train_test_split
X_tr, X_te, y_tr, y_te = train_test_split(X, y, test_size=0.1)
from keras.models import Model, Input
from keras.layers import LSTM, Embedding, Dense, TimeDistributed, Dropout, Bidirectional
input = Input(shape=(max_len,))
model = Embedding(input_dim=n_words, output_dim=50, input_length=max_len)(input)
model = Dropout(0.1)(model)
model = Bidirectional(LSTM(units=100, return_sequences=True, recurrent_dropout=0.1))(model)
out = TimeDistributed(Dense(n_tags, activation="softmax"))(model)  # softmax output layer
model = Model(input, out)
model.compile(optimizer="rmsprop", loss="categorical_crossentropy", metrics=["accuracy"])

In [32]:
import keras

In [33]:
import tensorflow as tf

In [39]:
history = model.fit(X_tr, np.array(y_tr), batch_size=32, epochs=100, validation_split=0.1, verbose=1)

#model.save('mmmodel.h5')  # creates a HDF5 file 'my_model.h5'
#del model  # deletes the existing model

# returns a compiled model
# identical to the previous one
#modelkk = load_model('mmmodel.h5')

Train on 48 samples, validate on 6 samples
Epoch 1/100
48/48 [==============================] - 0s 4ms/step - loss: 0.0425 - acc: 0.9925 - val_loss: 0.2413 - val_acc: 0.9433
Epoch 2/100
48/48 [==============================] - 0s 4ms/step - loss: 0.0403 - acc: 0.9942 - val_loss: 0.2285 - val_acc: 0.9533
Epoch 3/100
48/48 [==============================] - 0s 4ms/step - loss: 0.0374 - acc: 0.9929 - val_loss: 0.2340 - val_acc: 0.9567
Epoch 4/100
48/48 [==============================] - 0s 4ms/step - loss: 0.0368 - acc: 0.9946 - val_loss: 0.2291 - val_acc: 0.9533
Epoch 5/100
48/48 [==============================] - 0s 4ms/step - loss: 0.0338 - acc: 0.9962 - val_loss: 0.2304 - val_acc: 0.9533
Epoch 6/100
48/48 [==============================] - 0s 4ms/step - loss: 0.0326 - acc: 0.9967 - val_loss: 0.2277 - val_acc: 0.9567
Epoch 7/100
48/48 [==============================] - 0s 4ms/step - loss: 0.0307 - acc: 0.9963 - val_loss: 0.2315 - val_acc: 0.9533
Epoch 8/100
48/48 [=====================

Epoch 63/100
48/48 [==============================] - 0s 4ms/step - loss: 0.0025 - acc: 1.0000 - val_loss: 0.2528 - val_acc: 0.9667
Epoch 64/100
48/48 [==============================] - 0s 4ms/step - loss: 0.0023 - acc: 1.0000 - val_loss: 0.2567 - val_acc: 0.9600
Epoch 65/100
48/48 [==============================] - 0s 5ms/step - loss: 0.0029 - acc: 0.9996 - val_loss: 0.2572 - val_acc: 0.9633
Epoch 66/100
48/48 [==============================] - 0s 4ms/step - loss: 0.0023 - acc: 1.0000 - val_loss: 0.2585 - val_acc: 0.9633
Epoch 67/100
48/48 [==============================] - 0s 4ms/step - loss: 0.0021 - acc: 1.0000 - val_loss: 0.2554 - val_acc: 0.9600
Epoch 68/100
48/48 [==============================] - 0s 4ms/step - loss: 0.0026 - acc: 1.0000 - val_loss: 0.2539 - val_acc: 0.9667
Epoch 69/100
48/48 [==============================] - 0s 3ms/step - loss: 0.0020 - acc: 1.0000 - val_loss: 0.2506 - val_acc: 0.9633
Epoch 70/100
48/48 [==============================] - 0s 3ms/step - loss: 0.

In [40]:
test_sentence = "An individual wiping command should be implemented by setting the level of WW and INT-line from HIGH to LOW for the parameterized time p_t_einzelwischhub body control module".split(" ")

In [41]:
x_test_sent = pad_sequences(sequences=[[word2idx.get(w, 0) for w in test_sentence]],
                            padding="post", value=0, maxlen=max_len)

In [42]:
def newpredict(tt):
    result = []
    x_test_sent = pad_sequences(sequences=[[word2idx.get(w, 0) for w in tt]],
                            padding="post", value=0, maxlen=max_len)
    p = model.predict(np.array([x_test_sent[0]]))
    p = np.argmax(p, axis=-1)
    for w, pred in zip(tt, p[0]):
        result.append((w, tags[pred]))
    return result

In [43]:
newpredict(test_sentence)

[('An', 'Action'),
 ('individual', 'Action'),
 ('wiping', 'Action'),
 ('command', 'O'),
 ('should', 'Action'),
 ('be', 'O'),
 ('implemented', 'O'),
 ('by', 'O'),
 ('setting', 'O'),
 ('the', 'O'),
 ('level', 'O'),
 ('of', 'O'),
 ('WW', 'system_signal'),
 ('and', 'O'),
 ('INT-line', 'Action'),
 ('from', 'O'),
 ('HIGH', 'Action'),
 ('to', 'O'),
 ('LOW', 'Action'),
 ('for', 'O'),
 ('the', 'O'),
 ('parameterized', 'Action'),
 ('time', 'Action'),
 ('p_t_einzelwischhub', 'Action'),
 ('body', 'system component'),
 ('control', 'system component'),
 ('module', 'system component')]

In [ ]:
[('The', 'O'),
 ('system', 'Network_Component'),
 ('should', 'O'),
 ('do', 'O'),
 ('the', 'O'),
 ('validation', 'Action'),
 ('for', 'O'),
 ('confirm', 'Action'),
 ('password', 'Attribute'),
 ('text', 'O'),
 ('on', 'O'),
 ('submit', 'O'),
 ('of', 'O'),
 ('the', 'O'),
 ('form', 'O')]

In [44]:
scores = model.evaluate(X_tr, np.array(y_tr), verbose=1)
print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))

54/54 [==============================] - 0s 1ms/step
acc: 99.63%


In [45]:
# serialize model to JSON
model_json = model.to_json()
with open("/home/sharath/Documents/savedweights_Bilstm/modelpp6.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights("/home/sharath/Documents/savedweights_Bilstm/modelpp6.h5")
print("Saved model to disk")

Saved model to disk


In [46]:
from keras.models import model_from_json
# load json and create model
json_file = open('/home/sharath/Documents/savedweights_Bilstm/modelpp6.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)
# load weights into new model
loaded_model.load_weights("/home/sharath/Documents/savedweights_Bilstm/modelpp6.h5")
print("Loaded model from disk")

Loaded model from disk


In [28]:
import os
os.getcwd()

'/home/sharath'

In [47]:
loaded_model.compile(loss='categorical_crossentropy', optimizer='rmsprop', metrics=['accuracy'])

In [48]:
score = loaded_model.evaluate(X_tr, np.array(y_tr), verbose=1)
print("%s: %.2f%%" % (loaded_model.metrics_names[1], score[1]*100))

54/54 [==============================] - 1s 11ms/step
acc: 99.63%


In [49]:
import numpy as np

# Save
np.save('/home/sharath/Documents/savedweights_Bilstm/word2idx.npy', word2idx) 

# Load
#word2idx = np.load('C:\\Users\\sharath\\Desktop\\exp\\word2idx.npy').item()


# Save
np.save('/home/sharath/Documents/savedweights_Bilstm/tag2idx.npy', tag2idx) 

# Load
#tag2idx = np.load('C:\\Users\\sharath\\Desktop\\exp\\tag2idx.npy').item()

In [57]:
def bilstmfinal(test_sentence):
    import pandas as pd
    import numpy as np
    from keras.models import model_from_json
    from keras.preprocessing.sequence import pad_sequences
    from keras.models import Model, Input
    from keras.layers import LSTM, Embedding, Dense, TimeDistributed, Dropout, Bidirectional
    from sklearn.model_selection import train_test_split
    from keras.utils import to_categorical
    import numpy
    numpy.random.seed(7)
    word2idx = np.load('/home/sharath/Documents/savedweights_Bilstm/word2idx.npy').item()
    tag2idx = np.load('/home/sharath/Documents/savedweights_Bilstm/tag2idx.npy').item()
    # load json and create model
    json_file = open('/home/sharath/Documents/savedweights_Bilstm/modelpp6.json', 'r')
    loaded_model_json = json_file.read()
    json_file.close()
    loaded_model = model_from_json(loaded_model_json)
    # load weights into new model
    loaded_model.load_weights("/home/sharath/Documents/savedweights_Bilstm/modelpp6.h5")
    loaded_model.compile(loss='categorical_crossentropy', optimizer='rmsprop', metrics=['accuracy'])
    test_sentence = test_sentence.split(" ")
    x_test_sent = pad_sequences(sequences=[[word2idx.get(w, 0) for w in test_sentence]],
                            padding="post", value=0, maxlen=50)
    p = loaded_model.predict(np.array([x_test_sent[0]]))
    p = np.argmax(p, axis=-1)
    pls = []
    p = loaded_model.predict(np.array([x_test_sent[0]]))
    p = np.argmax(p, axis=-1)
    for w, pred in zip(test_sentence, p[0]):
        pls.append(pred)
    revsubs = { v:k for k,v in tag2idx.items()}
    entit = [revsubs.get(item,item) for item in pls]
    final = list(zip(test_sentence,entit))
    return final

In [58]:
s = "The system shall allow a registered visitor to load a custom filter."

In [59]:
bilstmfinal(s)

[('The', 'O'),
 ('architecture', 'O'),
 ('of', 'O'),
 ('Hatch', 'auxillary system'),
 ('washer', 'auxillary system'),
 ('consists', 'O'),
 ('of', 'O'),
 ('a', 'O'),
 ('gateway', 'system component'),
 ('connection', 'system component'),
 ('to', 'system component'),
 ('Controller', 'system component'),
 ('Area', 'system component'),
 ('Network', 'system component'),
 ('signal', 'system_signal'),
 ('BUS', 'system_signal'),
 ('and', 'O'),
 ('a', 'O'),
 ('combination', 'O'),
 ('BUS', 'system_signal')]

In [84]:
# def lstm_model():
#     import pandas as pd
#     import numpy as np
#     from keras.models import load_model


#     data = pd.read_csv('C:\\users\\sharath\\postags_v1.csv', encoding='latin1', sep = ',')
#     data = data.fillna(method="ffill")
#     words = list(set(data["Word"].values))
#     words.append("ENDPAD")
#     n_words = len(words)
#     tags = list(set(data["Tag"].values))
#     n_tags = len(tags)

#     class SentenceGetter(object):

#         def __init__(self, data):
#             self.n_sent = 1
#             self.data = data
#             self.empty = False
#             agg_func = lambda s: [(w, p, t) for w, p, t in zip(s["Word"].values.tolist(),
#                                                                s["POS"].values.tolist(),
#                                                                s["Tag"].values.tolist())]
#             self.grouped = self.data.groupby("Sentence #").apply(agg_func)
#             self.sentences = [s for s in self.grouped]

#         def get_next(self):
#             try:
#                 s = self.grouped["Sentence: {}".format(self.n_sent)]
#                 self.n_sent += 1
#                 return s
#             except:
#                 return None

#     getter = SentenceGetter(data)
#     sent = getter.get_next()
#     sentences = getter.sentences
#     max_len = 50
#     word2idx = {w: i for i, w in enumerate(words)}
#     tag2idx = {t: i for i, t in enumerate(tags)}
#     from keras.preprocessing.sequence import pad_sequences
#     X = [[word2idx[w[0]] for w in s] for s in sentences]
#     X = pad_sequences(maxlen=max_len, sequences=X, padding="post", value=n_words - 1)
#     y = [[tag2idx[w[2]] for w in s] for s in sentences]
#     y = pad_sequences(maxlen=max_len, sequences=y, padding="post", value=tag2idx["O"])
#     from keras.utils import to_categorical
#     y = [to_categorical(i, num_classes=n_tags) for i in y]
#     from sklearn.model_selection import train_test_split
#     X_tr, X_te, y_tr, y_te = train_test_split(X, y, test_size=0.1)
#     from keras.models import Model, Input
#     from keras.layers import LSTM, Embedding, Dense, TimeDistributed, Dropout, Bidirectional
#     input = Input(shape=(max_len,))
#     model = Embedding(input_dim=n_words, output_dim=50, input_length=max_len)(input)
#     model = Dropout(0.1)(model)
#     model = Bidirectional(LSTM(units=100, return_sequences=True, recurrent_dropout=0.1))(model)
#     out = TimeDistributed(Dense(n_tags, activation="softmax"))(model)  # softmax output layer
#     model = Model(input, out)
#     model.compile(optimizer="rmsprop", loss="categorical_crossentropy", metrics=["accuracy"])
#     history = model.fit(X_tr, np.array(y_tr), batch_size=32, epochs=10, validation_split=0.1, verbose=1)
#     history.model.save('m3odel.h5')  # creates a HDF5 file 'my_model.h5'
#     del model  # deletes the existing model
#     # returns a compiled model
#     # identical to the previous one
#     modelkk = load_model('m3odel.h5')
#     return modelkk

In [2]:
import tensorflow as tf

In [4]:
sess

In [8]:
tt = "The system shall allow the data manager to search occurrence resources by keywords"

In [9]:
tt = "The system shall allow the data manager to edit the metadata of an institution resource"